In [1]:
import numpy as np
import pandas as pd

In [38]:
### Source : Consommation alimentaire dans trois populations forestières de la région côtière du Cameroun: Yassa, Mvae et Bakola, page 490

muD = 1 / 60
m = 0.1

ConsoChasse = np.array([88, 70, 78])
ConsoLocal = np.array([10, 27, 18])
ConsoImportation = np.array([2,3,2])

Prop =  ConsoLocal / (ConsoChasse + ConsoImportation)
print(Prop)

fDs = Prop / (Prop + m) * muD
print("With proteins : ", fDs)
print("Average : ", np.average(fDs))
print(muD)

ConsoChasse = np.array([13, 12, 14])
ConsoLocal = np.array([82, 81, 81])
ConsoImportation = np.array([5,5,3])

Prop =  ConsoLocal / (ConsoChasse + ConsoImportation)
print(Prop)
fDs = Prop / (Prop + m) * muD
print("With calories : ", fDs)
print("Average : ", np.average(fDs))
print(muD)

[0.11111111 0.36986301 0.225     ]
With proteins :  [0.00877193 0.01311953 0.01153846]
Average :  0.011143308296906579
0.016666666666666666
[4.55555556 4.76470588 4.76470588]
With calories :  [0.01630867 0.01632406 0.01632406]
Average :  0.016318932398560002
0.016666666666666666


In [ ]:
### Source : DESA RAPPORT CFSAM 2019, région Est 2019

#### Culture ###
Culture = pd.DataFrame(np.array([[4840, 400], [18489, 19165], [30232, 20249], [840,105], [892657, 79175],[459043, 54343], [841323, 89106]]),
                       columns = ['Productions (T)', 'Superficies (Ha)'], index=["Igname", "Arachide", "Maïs", "Patate", "Banane Plantain", "Macabo", "Manioc"])

### Elevage ###
Elevage = pd.DataFrame(np.array([[6747],[492],[1206],[623],[2531]]),
                       columns = ['Prooductions(T)'], index=["Bovin", "Caprin", "Ovin", "Porcin", "Volailles"])

### Population ###
Pop = 771755



## Calcul de $r_F$
On utilise une formule donnant le taux de croissance intrinsèque à partir du poids, voir Relationship Among Body Mass, Metabolic Rate and the Intrinsic Rate of Natural Increase in Mammals.

J'applique cette formule au poids moyen des animaux chassés par une population Baka, voir Hunting Techniques, Wildlife Offtake and Market Integration. A Perspective from Individual Variations among the Baka (Cameroon)

In [31]:
wheightDruickerAll = 1897.7/206 * 1000 #poids des prises/ nbr des prises, en gramme
wheightRodentsAll = 509.6 / 254 * 1000
wheightHyraxes = 9 / 2 * 1000
wheightPholidotes = 38 / 16 *1000
wheightPrimatesAll = 596.8 / 71 * 1000
wheightCarbnivoresAll = 24.9 / 16 *1000
wheightBirdsAll = 5.13 / 6 *1000
wheightReptilesAll = 20.3 / 8 * 1000

allList = np.array([wheightDruickerAll, wheightRodentsAll, wheightHyraxes, wheightPholidotes, wheightPrimatesAll, wheightCarbnivoresAll, wheightBirdsAll, wheightReptilesAll])

rm = lambda w : 10**(-0.2622 * np.log10(w) + 0.6914)

print(rm(allList))
print(pd.DataFrame(rm(allList)).describe())


[0.44868827 0.66913207 0.54141175 0.64017838 0.45959746 0.71521529
 0.8368324  0.62916526]
              0
count  8.000000
mean   0.617528
std    0.131035
min    0.448688
25%    0.520958
50%    0.634672
75%    0.680653
max    0.836832


## Calcul de $K_F$
J'utilise les données issues de Ecological Structure of the Nonfiying Mammal Community at Cocha Cashu Biological Station, Manu National Park, Peru.

Elles donnent une densité d'animaux au km^2

Je multiplie ensuite cette densité par la zone de chasse. Celle ci varie, peu de données dessus (uniquement dans avila_interpreting_2019): 112 à 44 km2 pour 3 villages (la source de ces chiffres est introuvable).
Je prends (arbitrairement) 5 km^2

In [34]:
## Source : Ecological Structure of the Nonfiying Mammal Community at Cocha Cashu Biological Station, Manu National Park, Peru

density = [12,10,15,10,25,55,40,24,30,40,35,60,25,1,5,16,12,25,180,3.5,5.2,5.3,230,25.3,6.8,0.024,0.8,0.035,0.5,3,5.6,2.6]
densiteTotale = sum(density)
print(densiteTotale)
print(5*densiteTotale)
print(112/3*densiteTotale)

908.659
4543.295
33923.26933333334


## Calcul de $m_D$ et $m_W$
$1/m_W$ est estimé comme le temps passé à chasser.
Le temps passé à chasser varie selon les sources (15j/an pour avila_interpreting_2019, 160j/an pour duda_hunting_2017, 24/an pour jones_consequences_2020a, 5j/an pour jones_incentives_2019)

On obtient donc un intervalle de $[365/160, 365/5] = [2.25, 75]$.

On peut aussi écrire $m_W = \dfrac{\tilde{m_W}}{\epsilon}$ avec $\epsilon = 1/365$

Pour $m_D$, on utilise la relation $m = \dfrac{H_W^*}{H_D^*} = \dfrac{m_D}{m_W}$.
Les proportions de villageois/chasseurs sont calculées grâce aux données de avila_interpreting_2019


In [26]:
Population = pd.DataFrame(np.array([[82,71,85,143,163,152,98,81,71],[12,18,29,23,38,36,18,27,18]]),
                          columns=["DP2003", "DP2009", "DP2016", "M2003", "M2009", "M2016", "Mi2003", "Mi2009", "Mi2016"],
                          index=["Total Pop", "Total hunter"])
Population.loc["Pop villagers"] = Population.loc["Total Pop"] - Population.loc["Total hunter"] 
Population.loc["Prop Hunter / Total"] = Population.loc["Total hunter"] / Population.loc["Total Pop"]

Population.loc["Prop"] = Population.loc["Total hunter"] / Population.loc["Pop villagers"]
# print("Proportion : ", Population.loc["Prop"])
mean = Population.loc["Prop"].mean()

print("m_W min ", 365/160)
print("m_W max ", 365/5)
print("m = ", Population.loc["Prop"].describe())

print("max m_D : ", Population.loc["Prop"].max() * 365/5)
print("min m_D : ", Population.loc["Prop"].min() * 365/160)

m_W min  2.28125
m_W max  73.0
m =  count    9.000000
mean     0.322171
std      0.122573
min      0.171429
25%      0.225000
50%      0.310345
75%      0.339623
max      0.517857
Name: Prop, dtype: float64
max m_D :  37.80357142857143
min m_D :  0.39107142857142857


## Estimation de $f_D$

On estime $\mu_D$ comme étant le taux de mortalité.

On estime $f_D$ en utilisant la proportion de nourriture localement produite et celle importée/chassée. A l'équilibre, cette prop est égale à:
$$ r = \dfrac{f_D H_D}{e \lambda_F F_W^* H_W^*} = {f_D}{\mu_D - f_D} $$
et donc $f_D = \dfrac{r}{1+r} \mu_D$

L'intervalle de valeur de $r$ est estimé à partir de koppert_food_1993.

On prend $\mu_D = 1/50$.


In [39]:
print("Pour 50 ans)")
muD = 1/50
r = np.array([82/(18), 81/17, 81/17])
fD = r/(r+1) * muD
print("r", r)
print("muD : ", muD)
print(" fD : ", fD)


Pour 50 ans)
r [4.55555556 4.76470588 4.76470588]
muD :  0.02
 fD :  [0.0164     0.01653061 0.01653061]
